# Clean the data after correcting

## 1. Train a tokenizer with the cleaned corpus

The output folder will be the same, so the previous tokenizer will be overwritten.

In [1]:
import pandas as pd
from transformers import AutoTokenizer
import json
import nltk
nltk.download('punkt')

c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\santi\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [2]:
VOCAB_SIZE = 52000
TRAINING_BATCH_SIZE = 1000
HF_CHECKPOINT = "dccuchile/bert-base-spanish-wwm-cased"

pretrained_tokenizer = AutoTokenizer.from_pretrained(HF_CHECKPOINT)
df = pd.read_csv("../data/old-spanish-corpus-cleaned.tsv", sep='\t')

training_corpus = (
    df.loc[i:i+TRAINING_BATCH_SIZE, "text"].astype(str) 
    for i in range(0, len(df), TRAINING_BATCH_SIZE)
)

tokenizer = pretrained_tokenizer.train_new_from_iterator(training_corpus, VOCAB_SIZE)

C:\Users\santi\AppData\Local\Temp\ipykernel_1504\2841662301.py:6: DtypeWarning: Columns (1,2,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../data/old-spanish-corpus-cleaned.tsv", sep='\t')


In [3]:
example = df.loc[5, "text"]
print("BEFORE:", pretrained_tokenizer.tokenize(example)[:20])
print("AFTER:", tokenizer.tokenize(example)[:20])
tokenizer.save_pretrained("../output/tokenizer")

BEFORE: ['CON', '##QU', '##ISTA', '4', '*', '*', 'cu', '##tar', 'lo', 'que', 'fuese', 'de', 'su', 'mayor', 'agrado', ',', 'sin', 'dis', '"', 'curr']
AFTER: ['CONQUISTA', '4', '*', '*', 'cu', '##tar', 'lo', 'que', 'fuese', 'de', 'su', 'mayor', 'agrado', ',', 'sin', 'dis', '"', 'cur', '##rir', 'en']


('../output/tokenizer\\tokenizer_config.json',
 '../output/tokenizer\\special_tokens_map.json',
 '../output/tokenizer\\vocab.txt',
 '../output/tokenizer\\added_tokens.json',
 '../output/tokenizer\\tokenizer.json')

## 2. Chunk the texts that have more than 512 tokens

512 is the maximum number of tokens that can be processed by BERT-like models.

### 1. Chunk all the texts that have more than 512 tokens

In [4]:
MAX_TOKENS_LENGTH = 384 # 512*3/4, a number much below 512 so that it will still fit for different tokenizers

def chunk(text):
    chunks = []
    sentences = nltk.sent_tokenize(text) # text.split('.'), but enhanced
    current_chunk = sentences[0]
    for sentence in sentences[1:]:
        new_chunk_tks = len(tokenizer(f"{current_chunk} {sentence}")['input_ids'])
        if (new_chunk_tks) > (MAX_TOKENS_LENGTH-2):
            chunks.append(current_chunk)
            current_chunk = sentence
        else:
            current_chunk = f"{current_chunk} {sentence}"
    if current_chunk:
        chunks.append(current_chunk)
    return chunks

In [5]:
idx_max = dict()
for i, sentence in enumerate(df['text'].astype(str)):
    num_tokens = len(tokenizer(sentence)['input_ids'])
    if num_tokens > MAX_TOKENS_LENGTH:
        chunks = chunk(sentence)
        idx_max[i] = chunks
        # print(f"{i}. {num_tokens} tokens -> {len(chunks)} chunks")
        # print("Original:")
        # print(sentence)
        # print("Chunks: [")
        # [print(len(tokenizer(j)['input_ids']), j) for j in chunks]
        # print("]")
    if i % 100000 == 0:
        print(f"Chunked {i}/{len(df)} ({i/len(df):.2%})")

print("Finished 100%")

Chunked 0/1233487 (0.00%)


Token indices sequence length is longer than the specified maximum sequence length for this model (694 > 512). Running this sequence through the model will result in indexing errors


In [ ]:
original_length = len(df)
updated_length = len(df)+sum([len(i)-1 for i in idx_max.values()])
print(f"\nAdded {updated_length - original_length} chunks ({updated_length / original_length - 1:.2%})")


Added 235339 chunks (18.34%)


In [ ]:
assert df.loc[list(idx_max.keys())[0], "text"][:10] == idx_max[list(idx_max.keys())[0]][0][:10], "Texts do not coincide"

In [ ]:
with open("../data/chunkedTexts.json", "w") as json_file:
    json.dump(idx_max, json_file, indent=4)

#### 2. Update the original DataFrame with the chunked texts

In [ ]:
with open("../data/chunkedTexts.json", "r") as json_file:
    idx_max = json.load(json_file)
idx_max = {int(k): v for k, v in idx_max.items()}

In [ ]:
finaldf = []
for idx, row in df.iterrows():
    if idx in idx_max:
        cI = 0
        for t in idx_max[idx]:
            new_row = row.copy()
            new_row['text'] = t
            new_row['chunk_id'] = cI
            finaldf.append(new_row)
            cI += 1
    else:
        row['chunk_id'] = 0
        finaldf.append(row)
    if idx % 100000 == 0:
        print(f"Added {len(finaldf)}/{updated_length} ({len(finaldf)/updated_length:.2%})")

print("Finished 100%")
assert len(finaldf) == updated_length, f"Lengths must match, expected length to be {updated_length}, but was {len(finaldf)}"

Added 1/1518717 (0.00%)
Added 195369/1518717 (12.86%)
Added 367371/1518717 (24.19%)
Added 520011/1518717 (34.24%)
Added 621072/1518717 (40.89%)
Added 721348/1518717 (47.50%)
Added 823074/1518717 (54.20%)
Added 924267/1518717 (60.86%)
Added 1025951/1518717 (67.55%)
Added 1128348/1518717 (74.30%)
Added 1231229/1518717 (81.07%)
Added 1332768/1518717 (87.76%)
Added 1434333/1518717 (94.44%)
Finished 100%


In [ ]:
finaldf = pd.DataFrame(finaldf, columns=df.columns)
finaldf = finaldf.reset_index(drop=True)
finaldf = finaldf.reindex(columns=['source', 'source_id', 'source_text_id', 'chunk_id', 'title', 'date', 'place', 'text'])

finaldf

,source,source_id,source_id_text,title,date,place,text
0,The British Library,3436138,9,"Historia de la conquista de México, etc [With ...",1809-01-01 00:00:00,London,"HISTORIA DE LA CONQUISTA PE MÉXICO, POBLACIÓN ..."
1,The British Library,3436138,10,"Historia de la conquista de México, etc [With ...",1809-01-01 00:00:00,London,"This Work, as well as LAS FÁBULAS LITERARIAS, ..."
2,The British Library,3436138,11,"Historia de la conquista de México, etc [With ...",1809-01-01 00:00:00,London,"HISTORIA De la Conquista, población y Progreso..."
3,The British Library,3436138,12,"Historia de la conquista de México, etc [With ...",1809-01-01 00:00:00,London,"CONQUISTA quartel observando la batalla, y rec..."
4,The British Library,3436138,13,"Historia de la conquista de México, etc [With ...",1809-01-01 00:00:00,London,"DE NUEVA ESPAÑA. 3 "" pretexto á los sediciosos..."
...,...,...,...,...,...,...,...
1518712,Project Gutenberg,53294,917,Recuerdos Del Tiempo Viejo,1817-1893,?,Barcelona trabaja... y á su existencia ...
1518713,Project Gutenberg,53294,919,Recuerdos Del Tiempo Viejo,1817-1893,?,Olvidaba que entre ambas hay diferencia:...
1518714,Project Gutenberg,53294,920,Recuerdos Del Tiempo Viejo,1817-1893,?,La diferencia es esta: pero es preciso ...
1518715,Project Gutenberg,53294,921,Recuerdos Del Tiempo Viejo,1817-1893,?,"_Esta obra es propiedad de su Autor, el que pe..."


In [ ]:
finaldf.to_csv("../data/old-spanish-corpus-chunked.tsv", sep="\t", index=False)

In [ ]:
total_tokens = finaldf['text'].apply(lambda x: len(tokenizer.tokenize(x))).sum()
print(f"New dataset has:\n\ttokens: {total_tokens:,}\n\t   texts: {updated_length:,}")

New dataset has:
	tokens: 195,086,372
	   texts: 1,518,717
